# Understanding IfcRevolvedAreaSolid



In [ ]:
import ada
from ada.core.constants import X, Y, Z, O
from ada.core.curve_utils import get_center_from_3_points_and_radius
from ada.core.vector_utils import unit_vector, normal_to_points_in_plane, transform3d, EquationOfPlane, calc_yvec
from ada.core.utils import roundoff
from ada.occ.utils import make_eq_plane_object
import numpy as np

In [ ]:
def pipe_w_multiple_bends() -> ada.Pipe:
    z = 3.2
    y0 = -200e-3
    x0 = -y0
    coords = [
        (0, y0, z),
        (5 + x0, y0, z),
        (5 + x0, y0 + 5, z),
        (10, y0 + 5, z + 2),
        (10, y0 + 5, z + 10),
    ]
    pipe1 = ada.Pipe(
        "Pipe1",
        coords,
        ada.Section("PSec", "PIPE", r=0.10, wt=5e-3),
    )

    return ada.Assembly('TestModel') / ada.Part('TestSite') /  pipe1

a = pipe_w_multiple_bends()
pipe = list(a.get_all_physical_objects())[0]
pipe_bends = list(filter(lambda x: isinstance(x, ada.PipeSegElbow), pipe.segments))

in_plane_pipe_bend = pipe_bends[0]
out_of_plane_pipe_bend = pipe_bends[1]

## In-plane example

In [ ]:
pipe_elbow = in_plane_pipe_bend

xvec1 = unit_vector(pipe_elbow.xvec1)
xvec2 = unit_vector(pipe_elbow.xvec2)

p1, p2, p3 = pipe_elbow.p1.p, pipe_elbow.p2.p, pipe_elbow.p3.p

normal = normal_to_points_in_plane([p1, p2, p3])

cd = get_center_from_3_points_and_radius(p1, p2, p3, pipe_elbow.bend_radius)
extrusion_start_p = pipe_elbow.arc_seg.p1
diff = cd.center - extrusion_start_p

yvec = calc_yvec(xvec1, normal)
global_csys = (X, Y, Z)
new_csys = (normal, yvec, xvec1)

n_tra, diff_tra = transform3d(global_csys, new_csys, O, [normal, diff])

# Calculate back the transformed coordinate diff_tra_norm, which is defined in a local coordinate system "new_csys"
abs_coord = transform3d(new_csys, global_csys, O, [diff_tra])[0] + cd.center

# Is the axis point in the XY plane
eqp = EquationOfPlane(extrusion_start_p, normal)
eqp.is_point_in_plane(abs_coord)

In [ ]:
# a.add_part(make_eq_plane_object("InPlane", eqp))
# a.to_ifc('test.ifc')

In [ ]:
dist = diff.dot(eqp.normal)

extrusion_start_p, abs_coord, diff, dist

## Out-of-plane Example

In [ ]:
pipe_elbow = out_of_plane_pipe_bend

extrusion_start_p = pipe_elbow.arc_seg.p1

xvec1 = unit_vector(pipe_elbow.xvec1)

p1, p2, p3 = pipe_elbow.p1.p, pipe_elbow.p2.p, pipe_elbow.p3.p

normal = normal_to_points_in_plane([p1, p2, p3])
cd = get_center_from_3_points_and_radius(p1, p2, p3, pipe_elbow.bend_radius)
diff = np.array([roundoff(x) for x in cd.center - extrusion_start_p])

yvec = calc_yvec(xvec1, normal)
global_csys = (X, Y, Z)
new_csys = (normal, yvec, xvec1)

eqp = EquationOfPlane(extrusion_start_p, normal)
eqp.is_point_in_plane(abs_coord)

Okay, so the point is not in the XY-plane. How about projecting the point back into the XY-plane

In [ ]:
dist = diff.dot(eqp.normal)
proj_diff = diff - dist*eqp.normal

extrusion_start_p, cd.center, abs_coord, diff, dist, proj_diff